In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Embedding,Bidirectional,GRU,LSTM,Dense,Dropout,SpatialDropout1D
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.callbacks import EarlyStopping
from keras.preprocessing.text import Tokenizer

Read in `train.csv` which contains 2 columns:
1. df['Text']: Each row can be a huge chunk of text. The code below will split them into shorter sequences
2. df['Category']: One of prediction categories, e.g. Family, Criminal, etc

This `train.csv` can be really flexible, so you can plug any dataset you want into it. Due to time constraints, the dataset we used for the actual app was not large enough and not well-curated enough. However, after deployment it seemed to do okay, but definitely had edge cases and adversarial examples could easily be found.

In [ ]:
df = pd.read_csv('train.csv')

Text processing:
1. Remove non-alphabets
2. Make lowercase. Case-sensitive won't work well because in actual deployment, laymen probably don't care about typing with correct case
3. Split by spaces into a list
4. Stem words. Rationale: the dataset we used was small. But we need to be able to train on "divorce/divorces/divorced" and still predict based on "divorcee"
5. Join back the text into a big string. The tokenizing will occur later

In [ ]:
import re
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

X_processed = []
for text in df['Text']:
    text = re.sub('[^a-zA-Z]', ' ', text)
    text = text.lower()
    text = text.split()
    text = [ps.stem(word) for word in text]
    text = ' '.join(text)
    X_processed.append(text)

Fit, then use, the Keras Tokenizer on entire text corpus

In [ ]:
max_words = 20000
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_processed)
X_sequences = tokenizer.texts_to_sequences(X_processed)

In [ ]:
tokenizer_index_len = len(tokenizer.word_index)
print(tokenizer_index_len)

Split large sequences with 1000+ tokens into 20-length sequences
- Previously: ~150 samples, each a huge chunk of text
- After the next step: ~10,000 samples, each a fixed 20-length sequence

In [ ]:
sequence_len = 20
X = []
y_corpus = []

for i in range(len(X_sequences)):
    num_sequences = len(X_sequences[i]) // sequence_len
    
    for j in range(num_sequences):
        start = sequence_len * j
        end = sequence_len * (j + 1)
        X.append(X_sequences[i][start:end])
        y_corpus.append(df['Category'][i])

One-hot encode the Categories (e.g. Family, Criminal) because there is no 'ordering' of categories

In [ ]:
X = pd.DataFrame(X)

Y = pd.get_dummies(pd.Series(y_corpus))
Y = Y[ sorted(Y.columns) ]

print(X.shape)
print(Y.shape)

In [ ]:
print(list(Y.columns))

Perform train_test_split with a small 10% test set size

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.10, random_state = 42)
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

Create Deep Learning Model
- Train Embedding layer (but we really should have used GloVe pre-trained so it can generalize better in actual deployment)
- Bi-directional GRU model (bi-directional to capture context from both sides, GRU to balance training time and performance between RNN and LSTM
- Some SpatialDropout, dropout, recurrent_dropout for regularization because we need to use this DL model for actual deployment

In [ ]:
MAX_NB_WORDS = tokenizer_index_len + 1
EMBEDDING_DIM = 100

model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
# model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
# model.add(GRU(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Bidirectional(GRU(100, dropout=0.2, recurrent_dropout=0.2)))
# model.add(Dropout(0.2))
model.add(Dense(Y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', 
              metrics=['accuracy', tf.keras.metrics.TopKCategoricalAccuracy(k=2)])

Train model, with EarlyStopping

In [ ]:
epochs = 100
batch_size = 32

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,
                    validation_split=0.1,
                    callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)]
)

In [ ]:
# Evaluate the model
model.evaluate(X_test, Y_test)

Save both the model and tokenizer

In [ ]:
from tensorflow import keras
import pickle

model.save('saved_model')
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# END

Code below is old and not used.

In [ ]:
# plt.title('Loss')
# plt.plot(history.history['loss'], label='train')
# plt.plot(history.history['val_loss'], label='test')
# plt.legend()
# plt.show();

In [ ]:
# from keras.preprocessing.sequence import pad_sequences
# X = pad_sequences(X_sequences, maxlen=sequence_len)
# print(X.shape)

In [ ]:
# sequence_len = 20
# X_corpus = []
# y_corpus = []

# for i in range(len(df['Text'])):
#     text = df['Text'][i].split()
#     num_sequences = len(text) // sequence_len
    
#     for j in range(num_sequences):
#         start = sequence_len * j
#         end = sequence_len * (j + 1)
#         X_corpus.append(' '.join(text[start:end]))
#         y_corpus.append(df['Category'][i])

# from keras.preprocessing.text import Tokenizer
# max_words = 10000
# tokenizer = Tokenizer(num_words=max_words)
# tokenizer.fit_on_texts(X_corpus)
# X_sequences = tokenizer.texts_to_sequences(X_corpus)